In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [2]:
order_data = pd.read_csv('./OnlineRetail-clean.csv')

In [3]:
order_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,ImputedCustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,17850,United Kingdom


In [4]:
order_data['IsCancellation'] = order_data['InvoiceNo'].str.startswith('C')
order_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,ImputedCustomerID,Country,IsCancellation
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,17850,United Kingdom,False
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,17850,United Kingdom,False
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,17850,United Kingdom,False
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,17850,United Kingdom,False
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,17850,United Kingdom,False


In [5]:
len(order_data[order_data['IsCancellation']])

8704

In [6]:
len(order_data[~order_data['IsCancellation']])

527793

In [7]:
orders = order_data[~order_data['IsCancellation']]
cancels = order_data[order_data['IsCancellation']]
orders = orders.sample(n=len(cancels), random_state=12345)

order_data = pd.concat([orders, cancels], axis=0)
len(order_data)

17408

In [8]:
order_data = order_data.sample(frac=0.5, random_state=12345)
len(order_data)

8704

In [9]:
order_data.drop(['InvoiceNo', 'Description', 'Quantity','ImputedCustomerID'], axis=1, inplace=True)
order_data.head()

,StockCode,InvoiceDate,UnitPrice,CustomerID,Country,IsCancellation
52732,22617,2011-02-07 10:44:00,4.95,16686.0,United Kingdom,True
297318,22410,2011-10-17 13:31:00,2.46,14096.0,United Kingdom,False
73782,21843,2011-03-07 12:06:00,10.95,17139.0,United Kingdom,True
121369,22230,2011-05-04 11:23:00,0.85,13268.0,United Kingdom,False
135427,23172,2011-05-17 11:47:00,1.65,17677.0,United Kingdom,True


In [10]:
X, y = order_data.iloc[:, :-1], order_data.iloc[:, -1]
X.head()

,StockCode,InvoiceDate,UnitPrice,CustomerID,Country
52732,22617,2011-02-07 10:44:00,4.95,16686.0,United Kingdom
297318,22410,2011-10-17 13:31:00,2.46,14096.0,United Kingdom
73782,21843,2011-03-07 12:06:00,10.95,17139.0,United Kingdom
121369,22230,2011-05-04 11:23:00,0.85,13268.0,United Kingdom
135427,23172,2011-05-17 11:47:00,1.65,17677.0,United Kingdom


In [11]:
y

52732      True
297318    False
73782      True
121369    False
135427     True
          ...  
360955    False
249246    False
95775      True
349296     True
308688     True
Name: IsCancellation, Length: 8704, dtype: bool

In [12]:
X['InvoiceDate'] = pd.to_datetime(X['InvoiceDate'], format='%Y-%m-%d %H:%M:%S')

In [13]:
X['Month'] = X['InvoiceDate'].dt.month
X['Day'] = X['InvoiceDate'].dt.day
X['Hour'] = X['InvoiceDate'].dt.hour

In [14]:
X.drop('InvoiceDate', axis=1, inplace=True)
X.head()

,StockCode,UnitPrice,CustomerID,Country,Month,Day,Hour
52732,22617,4.95,16686.0,United Kingdom,2,7,10
297318,22410,2.46,14096.0,United Kingdom,10,17,13
73782,21843,10.95,17139.0,United Kingdom,3,7,12
121369,22230,0.85,13268.0,United Kingdom,5,4,11
135427,23172,1.65,17677.0,United Kingdom,5,17,11


In [15]:
dummy = pd.get_dummies(X['Country'])
dummy.head()

,Australia,Austria,Bahrain,Belgium,Canada,Channel Islands,Cyprus,Czech Republic,Denmark,EIRE,...,Poland,Portugal,RSA,Singapore,Spain,Sweden,Switzerland,USA,United Kingdom,Unspecified
52732,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
297318,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
73782,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
121369,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
135427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [16]:
X = pd.concat([X, dummy], axis=1).drop('Country', axis=1)
X.head()

,StockCode,UnitPrice,CustomerID,Month,Day,Hour,Australia,Austria,Bahrain,Belgium,...,Poland,Portugal,RSA,Singapore,Spain,Sweden,Switzerland,USA,United Kingdom,Unspecified
52732,22617,4.95,16686.0,2,7,10,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
297318,22410,2.46,14096.0,10,17,13,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
73782,21843,10.95,17139.0,3,7,12,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
121369,22230,0.85,13268.0,5,4,11,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
135427,23172,1.65,17677.0,5,17,11,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [17]:
dummy = pd.get_dummies(X['StockCode'])

In [18]:
X = pd.concat([X, dummy], axis=1).drop('StockCode', axis=1)
X.head()

,UnitPrice,CustomerID,Month,Day,Hour,Australia,Austria,Bahrain,Belgium,Canada,...,90204,90211B,90214B,90214C,90214H,90214M,90214N,90214Z,DCGSSBOY,gift_0001_20
52732,4.95,16686.0,2,7,10,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
297318,2.46,14096.0,10,17,13,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
73782,10.95,17139.0,3,7,12,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
121369,0.85,13268.0,5,4,11,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
135427,1.65,17677.0,5,17,11,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [20]:
orders_dmatrix = xgb.DMatrix(data=X, label=y)
params = {'objective':"binary:logistic", "max_depth":3}

In [21]:
cv_results = xgb.cv(dtrain=orders_dmatrix, params=params,
                    nfold=3, num_boost_round=5,
                    metrics="error", as_pandas=True, seed=12345)

In [22]:
print(cv_results)

   train-error-mean  train-error-std  test-error-mean  test-error-std
0          0.368164         0.011171         0.365234        0.008165
1          0.349035         0.004022         0.361559        0.008720
2          0.350643         0.007737         0.355815        0.012010
3          0.337718         0.001844         0.340074        0.004062
4          0.337603         0.005817         0.340650        0.014119


In [23]:
print("Accuracy:")
print(((1 - cv_results['test-error-mean']).iloc[-1]))

Accuracy:
0.6593500201495828


The model's accuracy is only 65%, making it only slightly better than random guessing. This model appears to be of limited utility.